In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import emcee

In [2]:
def log_likelihood(theta, frequency, logpower):
    alpha0, tau, gamma, alphaw = theta
    model = np.log10(rednoise(frequency, alpha0, tau, gamma, alphaw))
    return -0.5 * np.sum((logpower - model) ** 2)

In [3]:
def log_prior(theta):
    alpha0, tau, gamma, alphaw = theta
    if (alphaw < alpha0) & (tau > 1/365):
        return 0.0
    return -np.inf

In [4]:
def log_probability(theta, frequency, logpower):
    lp = log_prior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, frequency, logpower)

In [13]:
import emcee

#arg = fit_rednoise(time, mag)
arg = np.array([0.01, 10, 2, 1e-4])

pos = arg + 1e-4 * np.random.randn(32, 4)
nwalkers, ndim = pos.shape

sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability, args=(frequency, logpower))
sampler.run_mcmc(pos, 5000, progress=True);